In [1]:
from werkzeug.wrappers import Request, Response
#from flask import Flask, render_template
from jinja2 import Template

#from datetime import date
from datetime import datetime, timedelta

import collections
from amigocloud import AmigoCloud

from datetime import datetime

from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm
import requests

from docx2pdf import convert
import os

C:\Users\Bismar\.conda\envs\utea_reportes\lib\importlib\_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
C:\Users\Bismar\.conda\envs\utea_reportes\lib\importlib\_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
C:\Users\Bismar\.conda\envs\utea_reportes\lib\importlib\_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
C:\Users\Bismar\.conda\envs\utea_reportes\lib\importlib\_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)


In [2]:
# ruta de carpera donde estan las llaves
ruta = open('../../ruta.txt')
ruta = ruta.readlines()[0]

In [3]:
# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [8]:
fecha = '2023-05-10'

In [9]:
query_semana = {'query': 'select id, canhero from dataset_315614 where date(fecha_de_registro) = \'{fech}\' order by id'.format(fech=fecha)}
project = 'https://app.amigocloud.com/api/v1/projects/32920/sql'
select_semana = amigocloud.get(project, query_semana)
data_semana = select_semana['data']
data_semana

[{'id': 91, 'canhero': '8009 / ESPINOZA AGUILAR DAVID'},
 {'id': 92, 'canhero': '805 / AGROP. TRES HNOS DEL ORIENTE'},
 {'id': 93, 'canhero': '9677 / OVANDO JIMENEZ JAIRO ESTEVAN'},
 {'id': 95, 'canhero': '9677 / OVANDO JIMENEZ JAIRO ESTEVAN'},
 {'id': 96, 'canhero': '2220 / CRUZ VARGAS TITO JHONNY '},
 {'id': 97, 'canhero': '388 / AGROPECUARIA MARIANA S.R.L. '}]

In [10]:
for id in data_semana:
    print('ID inspeccion: ', id['id'])
    id_insp = id['id']
    # DATOS DE LAS INPECCION
    query = {'query': 'select * from dataset_315614 where id={id_insp}'.format(id_insp=id_insp)}
    project = 'https://app.amigocloud.com/api/v1/projects/32920/sql'
    select = amigocloud.get(project, query)
    data = select['data'][0]

    # convertir fecha a formato dd/mm/aaaa
    data['fecha_de_registro'] = datetime.strptime(data['fecha_de_registro'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')

    # los campos booleanos tiene formacto 'num_num', esto no es admitido para convertir a objeto
    # se modifica el dict para agregar 'k_' al inicio de todas llaves
    data = {'k_'+key : value for (key, value) in data.items()}
    insp = collections.namedtuple("insp", data.keys())(*data.values())
    
    # FOTOS
    query_fotos = {'query': 'select gal.s3_filename, fot.obs from dataset_315689 fot\
                                inner join gallery_48937 gal on fot.amigo_id=gal.source_amigo_id\
                                where fot.insp_ref_fot=\'{id}\''.format(id=insp.k_amigo_id)}
    project = 'https://app.amigocloud.com/api/v1/projects/32920/sql'
    fotos = amigocloud.get(project, query_fotos)['data']
    
    # CREAR REPORTE
    doc = DocxTemplate(ruta + '/templates/tpl_insp_cosechadoras_preventivo.docx')
    #descargar fotos y generar lista InlineImage
    lista_fotos_inline = []
    for foto in fotos:
        response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['s3_filename'])
        file = open(ruta + '/fotos/' + foto['s3_filename'], "wb")
        file.write(response.content)
        file.close()
        lista_fotos_inline.append({
            'foto': docxtpl.InlineImage(doc, image_descriptor= ruta + '/fotos/' + foto['s3_filename'], width=Mm(50)),
            'obs': foto['obs']
        })
    context = {'insp':insp, 'fotos':lista_fotos_inline}
    #context = {'insp':insp}
    doc.render(context)
    # formato de nombre de archivo: "123_CQ_01-01-2022_NOMBRE"
    cod_nom = insp.k_canhero.split(' / ')
    file_name = cod_nom[0] + '_ECP_' + insp.k_fecha_de_registro.replace('/','-') + '_' + cod_nom[1] + '_' + str(insp.k_id)
    #doc.save(r'C:\Users\UTEA\OneDrive - Ingenio Azucarero Guabira S.A (2)\INFORMES\IMFORMES PARA REVISAR\\' + file_name + '.docx')
    doc.save(ruta + r'/informes/_' + file_name + '.docx')
    print(file_name)
print('-------------------FIN')

ID inspeccion:  91
8009_ECP_10-05-2023_ESPINOZA AGUILAR DAVID_91
ID inspeccion:  92
805_ECP_10-05-2023_AGROP. TRES HNOS DEL ORIENTE_92
ID inspeccion:  93
9677_ECP_10-05-2023_OVANDO JIMENEZ JAIRO ESTEVAN_93
ID inspeccion:  95
9677_ECP_10-05-2023_OVANDO JIMENEZ JAIRO ESTEVAN_95
ID inspeccion:  96
2220_ECP_10-05-2023_CRUZ VARGAS TITO JHONNY _96
ID inspeccion:  97
388_ECP_10-05-2023_AGROPECUARIA MARIANA S.R.L. _97
-------------------FIN


In [21]:
lista_fotos_inline

[{'foto': <docxtpl.InlineImage at 0x2170850e808>, 'obs': 'Máquina lista '}]

## Convertir a PDF

In [11]:
path = r'' + ruta + '/informes'
contenido = os.listdir(path)
rutas =[]
i = 0
for name in contenido:
    rutas.append(path + '/' + name)
    i+=1

print('SE ENCONTRARON ' + str(i) + ' INFORMES')
print(len(rutas), 'INFORMES')
rutas

SE ENCONTRARON 6 INFORMES
6 INFORMES


['D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/2220_ECP_10-05-2023_CRUZ VARGAS TITO JHONNY _96.docx',
 'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/388_ECP_10-05-2023_AGROPECUARIA MARIANA S.R.L. _97.docx',
 'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/8009_ECP_10-05-2023_ESPINOZA AGUILAR DAVID_91.docx',
 'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/805_ECP_10-05-2023_AGROP. TRES HNOS DEL ORIENTE_92.docx',
 'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/9677_ECP_10-05-2023_OVANDO JIMENEZ JAIRO ESTEVAN_93.docx',
 'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/9677_ECP_10-05-2023_OVANDO JIMENEZ JAIRO ESTEVAN_95.docx']

In [14]:
i = 1
for filename in rutas:
    print('CONVIRTIENDO... INFORME ' + str(i) + ' ' + filename)
    # se realiza la convrsion pasando parametro de salida y entrada, 
    # para la salida se reemplaza la extencion y la ruta
    print(filename)
    print(filename.replace('.docx','.pdf').replace('informes','pdfs'))
    pdf = filename.replace('.docx','.pdf').replace('informes','pdfs').split('/')[-1]
    convert(filename, r'C:\Users\Bismar\Downloads\\' + pdf)
    i+=1
print('END')

CONVIRTIENDO... INFORME 1 D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/2220_ECP_10-05-2023_CRUZ VARGAS TITO JHONNY _96.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/2220_ECP_10-05-2023_CRUZ VARGAS TITO JHONNY _96.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdfs/2220_ECP_10-05-2023_CRUZ VARGAS TITO JHONNY _96.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

CONVIRTIENDO... INFORME 2 D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/388_ECP_10-05-2023_AGROPECUARIA MARIANA S.R.L. _97.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/388_ECP_10-05-2023_AGROPECUARIA MARIANA S.R.L. _97.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdfs/388_ECP_10-05-2023_AGROPECUARIA MARIANA S.R.L. _97.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

CONVIRTIENDO... INFORME 3 D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/8009_ECP_10-05-2023_ESPINOZA AGUILAR DAVID_91.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/8009_ECP_10-05-2023_ESPINOZA AGUILAR DAVID_91.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdfs/8009_ECP_10-05-2023_ESPINOZA AGUILAR DAVID_91.pdf


...

CONVIRTIENDO... INFORME 5 D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/9677_ECP_10-05-2023_OVANDO JIMENEZ JAIRO ESTEVAN_93.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/9677_ECP_10-05-2023_OVANDO JIMENEZ JAIRO ESTEVAN_93.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdfs/9677_ECP_10-05-2023_OVANDO JIMENEZ JAIRO ESTEVAN_93.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

CONVIRTIENDO... INFORME 6 D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/9677_ECP_10-05-2023_OVANDO JIMENEZ JAIRO ESTEVAN_95.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/9677_ECP_10-05-2023_OVANDO JIMENEZ JAIRO ESTEVAN_95.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdfs/9677_ECP_10-05-2023_OVANDO JIMENEZ JAIRO ESTEVAN_95.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

END
